# Appliances Energy Prediction

Machine Learning Regression Model to Predict Energy Efficiency of Buildings

In [1]:
# Import Libraries

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# Load Appliances Energy Prediction Data

energy_df = pd.read_csv('Dataset/energydata_complete.csv', parse_dates=['date'])
energy_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


## Dataset Information

https://archive.ics.uci.edu/ml/machine-learning-databases/00374

The dataset is Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. 

Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.

<table>
<thead><tr>
<th><strong>Feature Name</strong></th>
<th><strong>Description</strong></th>
<th><strong>Data Type</strong></th>
</tr>
</thead>
<tbody>
<tr>
<td>date</td>
<td>year-month-day hour:minute:second</td>
<td>datetime64[ns]</td>
</tr>
<tr>
<td>Appliances</td>
<td>energy use in Wh</td>
<td>int64</td>
</tr>
<tr>
<td>lights</td>
<td>energy use of light fixtures in the house in Wh</td>
<td>int64</td>
</tr>
<tr>
<td>T1</td>
<td>Temperature in kitchen area, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_1</td>
<td>Humidity in kitchen area, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T2</td>
<td>Temperature in living room area, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_2</td>
<td>Humidity in living room area, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T3</td>
<td>Temperature in laundry room area</td>
<td>float64</td>
</tr>
<tr>
<td>RH_3</td>
<td>Humidity in laundry room area, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T4</td>
<td>Temperature in office room, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_4</td>
<td>Humidity in office room, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T5</td>
<td>Temperature in bathroom, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_5</td>
<td>Humidity in bathroom, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T6</td>
<td>Temperature outside the building (north side), in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_6</td>
<td>Humidity outside the building (north side), in %</td>
<td>float64</td>
</tr>
<tr>
<td>T7</td>
<td>Temperature in ironing room , in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_7</td>
<td>Humidity in ironing room, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T8</td>
<td>Temperature in teenager room 2, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_8</td>
<td>Humidity in teenager room 2, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T9</td>
<td>Temperature in parents room, in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>RH_9</td>
<td>Humidity in parents room, in %</td>
<td>float64</td>
</tr>
<tr>
<td>T_out</td>
<td>Temperature outside (from Chievres weather station), in Celsius</td>
<td>float64</td>
</tr>
<tr>
<td>Press_mm_hg</td>
<td>Pressure (from Chievres weather station), in mm Hg</td>
<td>float64</td>
</tr>
<tr>
<td>RH_out</td>
<td>Humidity outside (from Chievres weather station), in %</td>
<td>float64</td>
</tr>
<tr>
<tr>
<td>Windspeed</td>
<td>Wind speed (from Chievres weather station), in m/s</td>
<td>float64</td>
</tr>
<tr>
<td>Visibility</td>
<td>Visibility (from Chievres weather station), in km</td>
<td>float64</td>
</tr>
<tr>
<td>Tdewpoint</td>
<td>Tdewpoint (from Chievres weather station), in °C</td>
<td>float64</td>
</tr>
<tr>
<td>rv1</td>
<td>Random variable 1, nondimensional</td>
<td>float64</td>
</tr>
<tr>
<td>rv2</td>
<td>Random variable 2, nondimensional</td>
<td>float64</td>
</tr>
<tr>
</tbody>
</table>

**Fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6).**

- What is the Root Mean Squared error in three D.P?

In [3]:
df_new = energy_df.drop(columns=['date'])

In [4]:
X = df_new['T2'].values.reshape(-1,1) # reshape to 2D array
y = df_new['T6']

In [5]:
linear_model = LinearRegression()
linear_model.fit(X, y)

# Making predictions
y_pred = linear_model.predict(X)

In [6]:
# Calculating RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
round(rmse, 3)

3.644

**Remove the following columns:**

[“date”, “lights”]

- The target variable is `Appliances`. 

- Use a 70-30 train-test set split with a  random state of 42 (for reproducibility). 

- Normalize the dataset using the MinMaxScaler (Hint: Use the MinMaxScaler fit_transform and transform methods on the train and test set respectively). 

- Run a multiple linear regression using the training set. 

**Answer the following questions:**

- What is the Mean Absolute Error (in three decimal places) for the  training set?

In [7]:
df_drop = energy_df.drop(columns=['date', 'lights'])
df_drop.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [8]:
X = df_drop.drop(columns=['Appliances'])
y = df_drop['Appliances']

In [9]:
# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [11]:
# Creating and fitting the model
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# Making predictions on the training set
y_train_pred = model.predict(x_train_scaled)

In [12]:
# Calculating MAE for the training set
mae_train = mean_absolute_error(y_train, y_train_pred)
round(mae_train, 3)

53.742

- What is the Root Mean Squared Error (in three decimal places) for the training set?

In [13]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
round(rmse_train, 3)

95.216

- What is the Mean Absolute Error (in three decimal places) for test set?

In [14]:
# Obtain predictions on the test set
predicted_values_scaled = model.predict(x_test_scaled)

In [15]:
mae = mean_absolute_error(y_test, predicted_values_scaled)
round(mae, 3)

53.643

- What is the Root Mean Squared Error (in three decimal places) for test set?

In [16]:
rmse = np.sqrt(mean_squared_error(y_test, predicted_values_scaled))
round(rmse, 3)

93.641

**Did the Model above overfit to the training set?**

- RMSE for training set: 95.216
- RMSE for test set: 93.641

No. Given that the RMSE on the test set is slightly lower than the RMSE on the training set, it suggests that the model generalizes well to unseen data. This is a positive sign as it indicates that the model is not overly fitting to the training data.

**Train a ridge regression model with default parameters.**

Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [17]:
ridge_reg = Ridge()
ridge_reg.fit(x_train_scaled, y_train)

Ridge()

In [18]:
# Make predictions on the test set
y_test_pred_ridge = ridge_reg.predict(x_test_scaled)

In [19]:
# Calculate RMSE for the test set
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))
round(rmse_ridge, 3)

93.709

Slight change from 93.641(linear regression) to 93.709(ridge regression). Tuning hyperparameters may improve ridge model.

**Train a lasso regression model with default value and obtain the new feature weights with it.**

How many of the features have non-zero feature weights?

In [20]:
lasso_reg = Lasso()
lasso_reg.fit(x_train_scaled, y_train)

Lasso()

In [21]:
# comparing the effects of regularisation

def get_weights_df(model, feat, col_name):

  # this function returns the weight of every feature

  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,rv1,-8.125910e+12,0.775954,-0.000000
1,RH_2,-4.695204e+02,-380.264856,-0.000000
2,T_out,-3.443917e+02,-220.546136,0.000000
3,T2,-2.527110e+02,-181.115240,0.000000
4,T9,-2.032405e+02,-198.646042,-0.000000
5,RH_8,-1.686277e+02,-166.246807,-26.102888
6,RH_out,-8.310943e+01,-36.798983,-50.293976
7,RH_7,-4.756344e+01,-50.215392,-0.000000
8,RH_9,-4.258525e+01,-45.543329,-0.000000
9,T5,-1.674855e+01,-25.703802,-0.000000


4 features have non-zero feature weights for lasso regression

**What is the new RMSE with the Lasso Regression on the test set?**

In [22]:
y_test_pred_lasso = lasso_reg.predict(x_test_scaled)

In [23]:
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
round(rmse_lasso, 3)

99.424